### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import pickle
import joblib
from boruta import BorutaPy
from scipy.stats import kendalltau
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_selection import SelectKBest, f_classif

In [9]:
df = pd.read_csv('first inten project.csv')
df.head()

Booking_ID  number of adults  number of children  number of weekend nights  \
0   INN00001                 1                   1                         2   
1   INN00002                 1                   0                         1   
2   INN00003                 2                   1                         1   
3   INN00004                 1                   0                         0   
4   INN00005                 1                   0                         1   

   number of week nights  type of meal  car parking space    room type  \
0                      5   Meal Plan 1                  0  Room_Type 1   
1                      3  Not Selected                  0  Room_Type 1   
2                      3   Meal Plan 1                  0  Room_Type 1   
3                      2   Meal Plan 1                  0  Room_Type 1   
4                      2  Not Selected                  0  Room_Type 1   

   lead time market segment type  repeated  P-C  P-not-C  average price   \
0        224             Offline         0    0        0           88.00   
1          5              Online         0    0        0          106.68   
2          1              Online         0    0        0           50.00   
3        211              Online         0    0        0          100.00   
4         48              Online         0    0        0           77.00   

   special requests date of reservation booking status  
0                 0           10/2/2015   Not_Canceled  
1                 1           11/6/2018   Not_Canceled  
2                 0           2/28/2018       Canceled  
3                 1           5/20/2017       Canceled  
4                 0           4/11/2018       Canceled

In [10]:
df['booking status'].value_counts()

Not_Canceled    24396
Canceled        11889
Name: booking status, dtype: int64

### Convert Categories

In [7]:
le = LabelEncoder()
for column in df.columns:
    if df[column].dtype == 'O':
        df[column] = le.fit_transform(df[column])

st = StandardScaler()
X = df.drop('booking status', axis=1)
X = st.fit_transform(X)
y = df['booking status']

### Grid Search Function

In [5]:
def best_parameters(X, y):
    param_grid = {'n_estimators': [60,80,100,120,140,160,180,200,220,240,260, 280], 'max_features': ['sqrt', 'log2']}
    grid_search = GridSearchCV(RandomForestClassifier(), param_grid)
    grid_search.fit(X,y)
    best_params = grid_search.best_params_
    print("Best Parameters: ", best_params)
    return best_params

### Feature Selection Function

In [22]:
def feature_selection(X, y, X1, type):
    if type == 'anova':
        # Perform feature selection using ANOVA
        selector = SelectKBest(f_classif, k=5)  # Select top 5 features
        X_new = selector.fit_transform(X, y)
        selected_feature_indices = selector.get_support(indices=True)
        selected_feature_names = X1.columns[selected_feature_indices]
        print("Selected feature names:", selected_feature_names)
        print("the number of features: ", len(selected_feature_names))
        return X_new, y
    if type == 'boruta':
        model = RandomForestClassifier()
        boruta_selector = BorutaPy(model, n_estimators='auto', verbose=0, random_state=1, max_iter=100, perc=85)
        boruta_selector.fit(X, y)
        print("Boruta Ranking: ", boruta_selector.ranking_)
        print("No. of significant features: ", boruta_selector.n_features_)
        selected_rf_features = pd.DataFrame({
            'Feature': list(X1.columns),
            'Ranking': boruta_selector.ranking_
        }).sort_values(by='Ranking')
        # Selecting the columns with ranking 1
        new_columns = selected_rf_features[selected_rf_features["Ranking"] == 1]['Feature']
        X_new = X1[new_columns]
        
        return X_new, y
    if type == 'kendall':
        correlations = {}
        for feature in X1.columns:
            tau, p_value = kendalltau(X1[feature], y)
            if p_value < 0.05:
                correlations[feature] = abs(tau)
        sorted_features = sorted(correlations.items(), key=lambda x: x[1], reverse=True)
        print("Number of Features: " + str(len(sorted_features)))
        n = 5
        selected_features = [f[0] for f in sorted_features[:n]]
        print("Selected Features:", selected_features)
        X_new = X1[selected_features]
        return X_new, y

### Apply Feature Selection

In [23]:
X1 = pd.DataFrame(X, columns=df.drop(['booking status'], axis=1).columns)
X_new, y_new = feature_selection(X, y, X1, type='kendall')
#new_X = filtered_data[selected_features]
print(X_new.shape, y_new.shape)
X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size=0.2, random_state=42)

Number of Features: 15
Selected Features: ['lead time', 'special requests', 'average price ', 'market segment type', 'repeated']
(36285, 5) (36285,)


In [29]:
X_new.columns

Index(['lead time', 'special requests', 'average price ',
       'market segment type', 'repeated'],
      dtype='object')

### Apply Grid Search

In [25]:
best_params = best_parameters(X_train, y_train)

Best Parameters:  {'max_features': 'sqrt', 'n_estimators': 280}


### Train Random Forest Classifier

In [26]:
rf = RandomForestClassifier(**best_params, random_state=42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

Accuracy: 0.864820173625465


### Convert Model for Deployment

In [ ]:
joblib.dump(rf, 'model_Kendall.pkl')

In [18]:
model = joblib.load('model_Anova.pkl')
model

RandomForestClassifier(max_features='log2', n_estimators=260, random_state=42)